In [119]:
import pandas as pd
import numpy as np
np.random.seed(192)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import lightgbm as lgbm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.impute import KNNImputer
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA


import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.tools import diff
%matplotlib inline
import statsmodels.api as sm
from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
import warnings
warnings.filterwarnings('ignore')
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR, ARResults, AutoReg
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from pmdarima import auto_arima
from statsmodels.tsa.arima_model import ARMA, ARMAResults, ARIMAResults
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.eval_measures import rmse
import lightgbm as lgbm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)

np.random.seed(192)
from pylab import rcParams
import scipy.stats as stats
rcParams['figure.figsize'] = (12,5)
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.labelsize'] = 12

In [120]:
# df7 = pd.read_pickle('master_data_top_80_res7.pkl')
# df14 = pd.read_pickle('master_data_top_80_res14.pkl')
df21 = pd.read_pickle('master_data_top_80_res21.pkl')
# df28 = pd.read_pickle('master_data_top_80_res28.pkl')
# df35 = pd.read_pickle('master_data_top_80_res35.pkl')
holiday_master = pd.read_pickle('holiday_master_5_1.pkl')
# df7 = pd.merge(df7, holiday_master, how='inner', 
#                on=['bkg_date', 'stc_lane']).copy()
# df14 = pd.merge(df14, holiday_master, how='inner', 
#                on=['bkg_date', 'stc_lane']).copy()
df21 = pd.merge(df21, holiday_master, how='inner', 
               on=['bkg_date', 'stc_lane']).copy()
# df28 = pd.merge(df28, holiday_master, how='inner', 
#                on=['bkg_date', 'stc_lane']).copy()
# df35 = pd.merge(df35, holiday_master, how='inner', 
#                on=['bkg_date', 'stc_lane']).copy()

In [121]:
def mape_score(y_actual, y_pred):
    score = np.round(np.mean(np.abs(y_actual-y_pred)/y_actual)*100,2)
    return score

def mda(actual, pred):
    """ Mean Directional Accuracy """
    result = np.mean((np.sign(actual.values[1:] - 
                              actual.values[:-1]) == np.sign(pred[1:] - pred[:-1])).astype(int))
    result = result*100
    
    return result

def mda1(actual, pred, dow_avg):
    """ Mean Directional Accuracy """
    result = np.mean((np.sign(actual.values- 
                              dow_avg) == np.sign(pred - dow_avg)).astype(int))
    result = result*100
    
    return result

In [122]:
# remove correlated features to reduce the feature space

def correlation(dataset, threshold):
    # set of all the names of correlated columns
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])>threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [123]:
top_80 = ['DELH_LKOD','BLRO_LKOD','DELH_MAAO','DELH_CCUO','AMDO_SLMO','DELH_NAGO','AMC_AMDO','AMC_SLMO','DELH_PNQO','DELH_IDRO',
 'DELH_BLRO','AMDO_BLRO','CCUO_DELH','AMC_CCUO','DELH_PATO','DELH_HYDO','BLRO_SLMO','BWDV_DELH','PNQO_AMDO','DELH_NDAO',
 'CCUO_SLMO','BLRO_MAAO','CCUO_BBIO','DELH_JAIO','DELH_AMC','DELH_GUAO','AMC_NDAO','AMC_PNQO','AMC_DELH','BLRO_PNQO',
 'PNQO_CCUO','AMDO_AMC','BLRO_NDAO','MAAO_HYDO','AMC_JAIO','DELH_BWDV','NDAO_AMC','BLRO_AMC','CCUO_PATO','AMC_BWDV',
 'NDAO_SLMO','AMDO_DELH','PNQO_BLRO','AMDO_CCUO','AMC_LKOD','AMDO_PNQO','DELH_AMDO','AMC_IDRO','AMC_BLRO','AMC_MAAO',
 'BWDV_BLRO','BLRO_DELH','PNQO_DELH','CCUO_PNQO','AMC_HYDO','PNQO_MAAO','AMC_PATO','AMC_NAGO','BLRO_JAIO','MAAO_SLMO',
 'DELH_SLMO','BLRO_VZAO','NDAO_CCUO','BLRO_AMDO','MAAO_VZAO','DELH_BBIO','CCUO_BLRO','BLRO_HYDO','NDAO_LKOD',
 'BLRO_CCUO','MAAO_AMDO','BWDV_PNQO','BWDV_AMDO','PNQO_AMC','PNQO_SLMO','SLMO_DELH','BWDV_CCUO','BWDV_AMC','MAAO_BLRO',
 'BWDV_SLMO','CCUO_GUAO','MAAO_DELH']

In [124]:
dow = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

In [125]:
def each_stc_preds1(df1):    
    count = 0
    d = []
    results = pd.DataFrame()
    for idx, v in enumerate(top_80):
        if(v=='CCUO_GUAO'):
            df = df1.loc[df1['stc_lane']==v].copy()
            df = df[df['bkg_date']>='2021-01-01'].copy()
        else:
            df = df1.loc[df1['stc_lane']==v].copy()
        stc = df[['stc_lane']].copy()
        bkg_date = df[['bkg_date']].copy()
        df.dropna(inplace=True)
        df.set_index('bkg_date', inplace=True)
        df.sort_index(inplace=True)
        df.drop(columns=['stc_lane'], inplace=True)    
        
        bar = (len(df)-8)
        bar1 = (len(df)-7)
        df['naive_preds'] = df['weight'].iloc[bar]
        df['avg_preds'] = df['weight'][:bar1].mean()
        df['sma_56d_preds'] = df['weight'].rolling(56).mean()
        df['sma_56d_preds'][bar1:] = df['sma_56d_preds'].iloc[bar]
        df['sma_28d_preds'] = df['weight'].rolling(28).mean()
        df['sma_28d_preds'][bar1:] = df['sma_28d_preds'].iloc[bar]
        df['sma_21d_preds'] = df['weight'].rolling(21).mean()
        df['sma_21d_preds'][bar1:] = df['sma_21d_preds'].iloc[bar]
        df['sma_14d_preds'] = df['weight'].rolling(14).mean()
        df['sma_14d_preds'][bar1:] = df['sma_14d_preds'].iloc[bar]
        df['sma_7d_preds'] = df['weight'].rolling(7).mean()
        df['sma_7d_preds'][bar1:] = df['sma_7d_preds'].iloc[bar]   
        
        preds = df[['naive_preds', 'avg_preds', 'sma_56d_preds', 'sma_28d_preds', 'sma_21d_preds',
                   'sma_14d_preds', 'sma_7d_preds']].copy()
        df.drop(columns = ['naive_preds', 'avg_preds', 'sma_56d_preds', 'sma_28d_preds', 'sma_21d_preds',
                   'sma_14d_preds', 'sma_7d_preds'], inplace=True)

        train_df = df.iloc[:-7]
        test_df = df.iloc[-7:]
        X_train = train_df.drop(labels=['weight'], axis=1)
        y_train = train_df['weight']
        X_valid = test_df.drop(labels=['weight'], axis=1)
        y_valid = test_df['weight']
        quasi_constant_feat = []
        for feature in X_train.columns:
            predominant = (X_train[feature].value_counts()/np.float(
            len(X_train))).sort_values(ascending=False).values[0]
            if predominant > 0.95:
                quasi_constant_feat.append(feature)
        X_train.drop(columns=quasi_constant_feat, inplace=True)
        X_valid.drop(columns=quasi_constant_feat, inplace=True)
        
        duplicated_feat = []
        for i in range(0, len(X_train.columns)):
            col_1 = X_train.columns[i]
            for col_2 in X_train.columns[i+1:]:
                if(X_train[col_1].equals(X_train[col_2])):
                    duplicated_feat.append(col_2)
        
        X_train.drop(columns=duplicated_feat, inplace=True)
        X_valid.drop(columns=duplicated_feat, inplace=True)
        
        corr_features = correlation(X_train, 0.75)
        X_train = X_train.drop(columns=corr_features)
        X_valid = X_valid.drop(columns=corr_features)
        inf_cols = X_train[X_train.columns[(X_train>100000).any()]].columns
        X_train = X_train.drop(columns=inf_cols)
        X_valid = X_valid.drop(columns=inf_cols)
        train_d = pd.concat([X_train, y_train], axis=1)
        valid_d = pd.concat([X_valid, y_valid], axis=1)
        df_d = pd.concat([train_d, valid_d])
        df_d = pd.concat([df_d, bkg_date], axis=1)
#         df_d.set_index('bkg_date', inplace=True)
#         df_d.sort_index(inplace=True)
        final = pd.DataFrame()
        for idx_d, v_dow in enumerate(dow):
            df_d1 = df_d.loc[df_d['bkg_date_dayofweek']==v_dow].copy()
            bkg_date = df_d1[['bkg_date']].copy()
            train_df = df_d1.iloc[:-1]
            test_df = df_d1.iloc[-1:]
            X_train = train_df.drop(labels=['weight', 'bkg_date'], axis=1)
            y_train = train_df['weight']
            X_valid = test_df.drop(labels=['weight', 'bkg_date'], axis=1)
            y_valid = test_df['weight']
            model_full = RandomForestRegressor(random_state=192)
            model_full.fit(X_train, y_train)
            y_pred_train = model_full.predict(X_train)
            y_pred_valid = model_full.predict(X_valid)
            X_train['rf_preds'] = y_pred_train
            X_valid['rf_preds'] = y_pred_valid
            X_train = pd.DataFrame(X_train)
            X_valid = pd.DataFrame(X_valid)
            train_data = pd.concat([X_train, y_train], axis=1)
            valid_data = pd.concat([X_valid, y_valid], axis=1)
            df_f = pd.concat([train_data, valid_data])
            df_f = pd.concat([df_f, bkg_date], axis=1)
            final = pd.concat([final, df_f], axis=0).reset_index(drop=True)
        

        final.index = preds.index
        #print('preds shape', preds.shape, 'final shape', final.shape)
        final = pd.concat([final, preds], axis=1)
        final.sort_index(inplace=True)
        stc.index = final.index
        #bkg_date.index = final.index
        #print('stc shape', stc.shape, 'bkg date shape', bkg_date.shape)
        final_1 = pd.concat([final, stc], axis=1)
        #final_1 = pd.concat([final_1, bkg_date], axis=1)
        #print('final_1 shape', final_1.shape)
        final = pd.DataFrame(final_1)
#         final.set_index('bkg_date', inplace=True)
#         final.sort_index(inplace=True)
        results = pd.concat([results, final], axis=0).reset_index(drop=True)
        print('shape of returned data ', results.shape)
        count = count+1
        print('Running on ', count, 'lane out of ', len(top_80))
        
        train = final.iloc[:-7]
        test = final.iloc[-7:]
        #X_train = train_df.drop(labels=['weight'], axis=1)
        y_train = train['weight']
        y_pred_train = train['rf_preds']
        #X_valid = test_df.drop(labels=['weight'], axis=1)
        y_valid = test['weight']
        y_pred_valid = test['rf_preds']

        train_mape_rf = mape_score(y_train, y_pred_train)
        test_mape_rf = mape_score(y_valid, y_pred_valid)
        #test_mda_rf = mda(y_valid, y_pred_valid)
        #test_mda14_rf = mda1(y_valid, y_pred_valid, dow_avg)
        test_mape_naive = mape_score(y_valid.values, final['naive_preds'].iloc[(len(final)-7):].values)
        #test_mda_naive = mda(y_valid, final['naive_preds'].iloc[bar1:].values)
        #test_mda14_naive = mda1(y_valid, final['naive_preds'].iloc[bar1:].values, dow_avg)
        test_mape_avg = mape_score(y_valid.values, final['avg_preds'][bar1:].values)
        #test_mda_avg = mda(y_valid, final['avg_preds'][bar1:].values)
        #test_mda14_avg = mda1(y_valid, final['avg_preds'][bar1:].values, dow_avg)
        test_mape_sma_56 = mape_score(y_valid.values, final['sma_56d_preds'][bar1:].values)
        #test_mda_sma_56 = mda(y_valid, final['sma_56d_preds'][bar1:].values)
       # test_mda14_sma_56 = mda1(y_valid, final['sma_56d_preds'][bar1:].values, dow_avg)
        test_mape_sma_28 = mape_score(y_valid.values, final['sma_28d_preds'][bar1:].values)
        #test_mda_sma_28 = mda(y_valid, final['sma_28d_preds'][bar1:].values)
        #test_mda14_sma_28 = mda1(y_valid, final['sma_28d_preds'][bar1:].values, dow_avg)
        test_mape_sma_21 = mape_score(y_valid.values, final['sma_21d_preds'][bar1:].values)
        #test_mda_sma_21 = mda(y_valid, final['sma_21d_preds'][bar1:].values)
        #test_mda14_sma_21 = mda1(y_valid, final['sma_21d_preds'][bar1:].values, dow_avg)
        test_mape_sma_14 = mape_score(y_valid.values, final['sma_14d_preds'][bar1:].values)
        #test_mda_sma_14 = mda(y_valid, final['sma_14d_preds'][bar1:].values)
        test_mape_sma_7 = mape_score(y_valid.values, final['sma_7d_preds'][bar1:].values)
        #test_mda_sma_7 = mda(y_valid, final['sma_7d_preds'][bar1:].values)
        #test_mda14_sma_7 = mda1(y_valid, final['sma_7d_preds'][bar1:].values, dow_avg)
        test_mape_lag_7 = mape_score(y_valid.values, final['weight_lag7'][bar1:].values)
        #test_mda_lag_7 = mda(y_valid, final['weight_lag7'][bar1:].values)
        #test_mda14_lag_7 = mda1(y_valid, final['weight_lag7'][bar1:].values, dow_avg)
        test_mape_lag_14 = mape_score(y_valid.values, final['weight_lag14'][bar1:].values)
        #test_mda_lag_14 = mda(y_valid, final['weight_lag14'][bar1:].values)
        #test_mda14_lag_14 = mda1(y_valid, final['weight_lag14'][bar1:].values, dow_avg)
        test_mape_lag_21 = mape_score(y_valid.values, final['weight_lag21'][bar1:].values)
        #test_mda_lag_21 = mda(y_valid, final['weight_lag21'][bar1:].values)
        #test_mda14_lag_21 = mda1(y_valid, final['weight_lag21'][bar1:].values, dow_avg)
        test_mape_lag_30 = mape_score(y_valid.values, final['weight_lag30'][bar1:].values)
        #test_mda_lag_30 = mda(y_valid, final['weight_lag30'][bar1:].values)
        #test_mda14_lag_30 = mda1(y_valid, final['weight_lag30'][bar1:].values, dow_avg)
        test_mape_lag_90 = mape_score(y_valid.values, final['weight_lag90'][bar1:].values)
        #test_mda_lag_90 = mda(y_valid, final['weight_lag90'][bar1:].values)
        #test_mda14_lag_90 = mda1(y_valid, final['weight_lag90'][bar1:].values, dow_avg)
        test_mape_lag_365 = mape_score(y_valid.values, final['weight_lag365'][bar1:].values)
        #test_mda_lag_365 = mda(y_valid, final['weight_lag365'][bar1:].values)
        #test_mda14_lag_365 = mda1(y_valid, final['weight_lag365'][bar1:].values, dow_avg)
        
        d.append({
            'stc_lane': v,
            'train_accuracy': (100 - train_mape_rf),
            'train_weight_sum': (y_train.sum()),
            'test_accuracy_rf': (100 - test_mape_rf),
            'test_accuracy_naive': (100 - test_mape_naive),
            'test_accuracy_avg': (100 - test_mape_avg),
            'test_accuracy_sma_56': (100 - test_mape_sma_56),
            'test_accuracy_sma_28': (100 - test_mape_sma_28),
            'test_accuracy_sma_21': (100 - test_mape_sma_21),
            'test_accuracy_sma_14': (100 - test_mape_sma_14),
            'test_accuracy_sma_7': (100 - test_mape_sma_7),
            'test_accuracy_lag_7': (100- test_mape_lag_7),
            'test_accuracy_lag_14': (100- test_mape_lag_14),
            'test_accuracy_lag_21': (100- test_mape_lag_21),
            'test_accuracy_lag_30': (100- test_mape_lag_30),
            'test_accuracy_lag_90': (100- test_mape_lag_90),
            'test_accuracy_lag_365': (100- test_mape_lag_365),
                        
#             'test_mda_accuracy_rf': np.ceil(test_mda14_rf),
#             'test_mda_accuracy_naive': np.ceil(test_mda14_naive),
#             'test_mda_accuracy_avg': np.ceil(test_mda14_avg),
#             'test_mda_accuracy_sma_56': np.ceil(test_mda14_sma_56),
#             'test_mda_accuracy_sma_28': np.ceil(test_mda14_sma_28),
#             'test_mda_accuracy_sma_21': np.ceil(test_mda14_sma_21),
# #             'test_mda14_accuracy_sma_14': np.ceil(test_mda_sma_14),
#             'test_mda_accuracy_sma_7': np.ceil(test_mda14_sma_7),
#             'test_mda_accuracy_lag_7': np.ceil(test_mda14_lag_7),
#             'test_mda_accuracy_lag_14': np.ceil(test_mda14_lag_14),
#             'test_mda_accuracy_lag_21': np.ceil(test_mda14_lag_21),
#             'test_mda_accuracy_lag_30': np.ceil(test_mda14_lag_30),
#             'test_mda_accuracy_lag_90': np.ceil(test_mda14_lag_90),
#             'test_mda_accuracy_lag_365': np.ceil(test_mda14_lag_365),

            'test_weight_sum': (y_valid.sum())
        })
        
    return results, d

In [126]:
%%time
# df7_p, res7 = each_stc_preds1(df7)
# df14_p, res14 = each_stc_preds1(df14)
df21_p, res21 = each_stc_preds1(df21)
# df28_p, res28 = each_stc_preds1(df28)
# df35_p, res35 = each_stc_preds1(df35)

shape of returned data  (1417, 69)
Running on  1 lane out of  82
shape of returned data  (2834, 74)
Running on  2 lane out of  82
shape of returned data  (4251, 88)
Running on  3 lane out of  82
shape of returned data  (5668, 89)
Running on  4 lane out of  82
shape of returned data  (7085, 94)
Running on  5 lane out of  82
shape of returned data  (8502, 95)
Running on  6 lane out of  82
shape of returned data  (9919, 97)
Running on  7 lane out of  82
shape of returned data  (11336, 97)
Running on  8 lane out of  82
shape of returned data  (12753, 97)
Running on  9 lane out of  82
shape of returned data  (14170, 99)
Running on  10 lane out of  82
shape of returned data  (15587, 99)
Running on  11 lane out of  82
shape of returned data  (17004, 100)
Running on  12 lane out of  82
shape of returned data  (18421, 101)
Running on  13 lane out of  82
shape of returned data  (19838, 101)
Running on  14 lane out of  82
shape of returned data  (21255, 101)
Running on  15 lane out of  82
shape o

In [127]:
# res7 = pd.DataFrame(res7)
# res14 = pd.DataFrame(res14)
res21 = pd.DataFrame(res21)
# res28 = pd.DataFrame(res28)
# res35 = pd.DataFrame(res35)
# res7.columns = res7.columns.values + '_7'
# res14.columns = res14.columns.values + '_14'
res21.columns = res21.columns.values + '_21'
# res28.columns = res28.columns.values + '_28'
# res35.columns = res35.columns.values + '_35'
# res7.rename(columns={'stc_lane_7': 'stc_lane'}, inplace=True)
# res14.rename(columns={'stc_lane_14': 'stc_lane'}, inplace=True)
res21.rename(columns={'stc_lane_21': 'stc_lane'}, inplace=True)
# res28.rename(columns={'stc_lane_28': 'stc_lane'}, inplace=True)
# res35.rename(columns={'stc_lane_35': 'stc_lane'}, inplace=True)
# res7.set_index('stc_lane',inplace=True)
# res14.set_index('stc_lane',inplace=True)
res21.set_index('stc_lane',inplace=True)
# res28.set_index('stc_lane', inplace=True)
# res35.set_index('stc_lane', inplace=True)
resz = pd.concat([res21#,res14,res21, res28, res35
                 ],axis=1,sort=False).reset_index()
resz.rename(columns = {'index':'stc_lane'}, inplace=True)
res = resz.copy()
# res['max_value_7'] = res[['test_accuracy_rf_7', 'test_accuracy_naive_7', 'test_accuracy_avg_7', 'test_accuracy_sma_56_7',
#                    'test_accuracy_sma_28_7', 'test_accuracy_sma_21_7', 'test_accuracy_sma_14_7',
#        'test_accuracy_sma_7_7', 'test_accuracy_lag_7_7', 'test_accuracy_lag_14_7', 'test_accuracy_lag_21_7',
#                          'test_accuracy_lag_30_7', 'test_accuracy_lag_90_7', 'test_accuracy_lag_365_7']].max(axis=1)
# res['max_value_14'] = res[['test_accuracy_rf_14', 'test_accuracy_naive_14', 'test_accuracy_avg_14', 'test_accuracy_sma_56_14',
#                    'test_accuracy_sma_28_14', 'test_accuracy_sma_21_14', 'test_accuracy_sma_14_14',
#        'test_accuracy_sma_7_14', 'test_accuracy_lag_7_14', 'test_accuracy_lag_14_14', 'test_accuracy_lag_21_14',
#                          'test_accuracy_lag_30_14', 'test_accuracy_lag_90_14', 'test_accuracy_lag_365_14']].max(axis=1)
res['max_value_21'] = res[['test_accuracy_rf_21', 'test_accuracy_naive_21', 'test_accuracy_avg_21', 'test_accuracy_sma_56_21',
                   'test_accuracy_sma_28_21', 'test_accuracy_sma_21_21', 'test_accuracy_sma_14_21',
       'test_accuracy_sma_7_21', 'test_accuracy_lag_7_21', 'test_accuracy_lag_14_21', 'test_accuracy_lag_21_21',
                         'test_accuracy_lag_30_21', 'test_accuracy_lag_90_21', 'test_accuracy_lag_365_21']].max(axis=1)
# res['max_value_28'] = res[['test_accuracy_rf_28', 'test_accuracy_naive_28', 'test_accuracy_avg_28', 'test_accuracy_sma_56_28',
#                    'test_accuracy_sma_28_28', 'test_accuracy_sma_21_28', 'test_accuracy_sma_14_28',
#        'test_accuracy_sma_7_28', 'test_accuracy_lag_7_28', 'test_accuracy_lag_14_28', 'test_accuracy_lag_21_28',
#                          'test_accuracy_lag_30_28', 'test_accuracy_lag_90_28', 'test_accuracy_lag_365_28']].max(axis=1)
# res['max_value_35'] = res[['test_accuracy_rf_35', 'test_accuracy_naive_35', 'test_accuracy_avg_35', 'test_accuracy_sma_56_35',
#                    'test_accuracy_sma_28_35', 'test_accuracy_sma_21_35', 'test_accuracy_sma_14_35',
#        'test_accuracy_sma_7_35', 'test_accuracy_lag_7_35', 'test_accuracy_lag_14_35', 'test_accuracy_lag_21_35',
#                          'test_accuracy_lag_30_35', 'test_accuracy_lag_90_35', 'test_accuracy_lag_365_35']].max(axis=1)

# res['max_value'] = res[['max_value_7'#, 'max_value_14', 'max_value_21', 'max_value_28', 'max_value_35'
#                        ]].max(axis=1)
res.sort_values(by='max_value_21', ascending=False, inplace=True)
res

,stc_lane,train_accuracy_21,train_weight_sum_21,test_accuracy_rf_21,test_accuracy_naive_21,test_accuracy_avg_21,test_accuracy_sma_56_21,test_accuracy_sma_28_21,test_accuracy_sma_21_21,test_accuracy_sma_14_21,test_accuracy_sma_7_21,test_accuracy_lag_7_21,test_accuracy_lag_14_21,test_accuracy_lag_21_21,test_accuracy_lag_30_21,test_accuracy_lag_90_21,test_accuracy_lag_365_21,test_weight_sum_21,max_value_21
19,DELH_NDAO,87.09,24804.0,89.96,-65.95,-37.25,-87.78,-96.47,-113.43,-110.04,-49.00,31.37,43.79,76.27,-93.08,-42.63,-79.24,73.0,89.96
1,BLRO_LKOD,85.50,16593.0,88.61,-62.73,-57.21,-59.21,-51.00,-46.05,-43.96,6.03,29.23,46.14,54.84,-121.01,-92.09,-118.79,39.0,88.61
24,DELH_AMC,85.17,49628.0,88.50,-244.28,-89.65,-123.52,-138.76,-156.26,-165.80,-119.14,27.96,9.16,47.44,-154.90,-107.57,-118.24,137.0,88.50
65,DELH_BBIO,84.06,17048.0,88.29,-156.19,-75.59,-130.96,-159.12,-181.54,-195.68,-188.37,15.95,12.38,39.52,-148.33,-105.24,-137.62,45.0,88.29
48,AMC_BLRO,84.41,25207.0,88.25,-141.35,-76.99,-110.87,-113.64,-125.83,-151.32,-105.88,61.19,47.26,65.28,-127.70,-60.95,-175.60,50.0,88.25
4,AMDO_SLMO,83.52,18023.0,87.91,-166.43,-46.10,-66.21,-79.64,-89.29,-90.66,-78.27,64.52,46.43,60.00,-116.90,-77.38,-79.76,43.0,87.91
8,DELH_PNQO,85.57,26233.0,87.19,7.02,18.74,-35.11,-42.67,-45.13,-46.03,-23.51,58.87,50.64,74.36,-74.02,39.27,-13.11,84.0,87.19
0,DELH_LKOD,84.04,41459.0,86.97,-199.51,-148.95,-178.27,-176.91,-184.96,-197.95,-140.28,1.35,22.84,71.05,-160.86,-142.97,-173.77,95.0,86.97
13,AMC_CCUO,84.89,20708.0,86.37,-38.91,-69.41,-62.21,-65.17,-76.54,-100.61,-73.04,26.27,17.27,38.14,-60.63,-79.92,-189.13,44.0,86.37
32,BLRO_NDAO,84.29,17119.0,85.67,-46.94,-93.32,-82.03,-94.51,-99.97,-112.45,-59.42,47.82,42.11,68.91,-135.31,-92.65,-204.01,38.0,85.67


* low performing lanes for df7(last week of august) - NDAO_LKOD, NDAO_CCUO, CCUO_BLRO, NDAO_AMC, SLMO_DELH
* for df 14 there were higher number of lanes (in compare to df7) with negative accuracy - AMC_BWDV, BWDV_PNQO, PNQO_MAAO, BWDV_DELH, PNQO_AMDO, BWDV_AMDO, BWDV_AMC, BWDV_CCUO, PNQO_AMC, BWDV_BLRO, CCUO_PNQO, AMDO_DELH, MAAO_BLRO, MAAO_SLMO
* low performing lanes for df 21 - PNQO_DELH, CCUO_DELH, AMC_NAGO
* for df 28 there were higher number of lanes (in compare to df7) with negative accuracy - PNQO_DELH, BLRO_MAAO, PNQO_AMDO, MAAO_BLRO, BWDV_SLMO, MAAO_SLMO, BWDV_DELH, BLRO_DELH
* low performing lane for df35 - SLMO_DELH

In [128]:
# # res = resz.copy()
# res['max_mda_value_7'] = res[['test_mda_accuracy_rf_7', 'test_mda_accuracy_naive_7', 'test_mda_accuracy_avg_7', 'test_mda_accuracy_sma_56_7',
#                    'test_mda_accuracy_sma_28_7', 'test_mda_accuracy_sma_21_7',# 'test_mda_accuracy_sma_14_7',
#        'test_mda_accuracy_sma_7_7', 'test_mda_accuracy_lag_7_7', 'test_mda_accuracy_lag_14_7', 'test_mda_accuracy_lag_21_7',
#                          'test_mda_accuracy_lag_30_7', 'test_mda_accuracy_lag_90_7', 'test_mda_accuracy_lag_365_7']].max(axis=1)

# res['max_value'] = res[['test_mda_accuracy_rf_7', 'test_mda_accuracy_naive_7', 'test_mda_accuracy_avg_7', 'test_mda_accuracy_sma_56_7',
#                    'test_mda_accuracy_sma_28_7', 'test_mda_accuracy_sma_21_7',# 'test_mda_accuracy_sma_14_7',
#        'test_mda_accuracy_sma_7_7', 'test_mda_accuracy_lag_7_7', 'test_mda_accuracy_lag_14_7', 'test_mda_accuracy_lag_21_7',
#                          'test_mda_accuracy_lag_30_7', 'test_mda_accuracy_lag_90_7', 'test_mda_accuracy_lag_365_7']].idxmax(axis="columns")

# # res['max_mda_value_14'] = res[['test_mda_accuracy_rf_14', 'test_mda_accuracy_naive_14', 'test_mda_accuracy_avg_14', 'test_mda_accuracy_sma_56_14',
# #                    'test_mda_accuracy_sma_28_14', 'test_mda_accuracy_sma_21_14', 'test_mda_accuracy_sma_14_14',
# #        'test_mda_accuracy_sma_7_14', 'test_mda_accuracy_lag_7_14', 'test_mda_accuracy_lag_14_14', 'test_mda_accuracy_lag_21_14',
# #                          'test_mda_accuracy_lag_30_14', 'test_mda_accuracy_lag_90_14', 'test_mda_accuracy_lag_365_14']].max(axis=1)
# # res['max_mda_value_21'] = res[['test_mda_accuracy_rf_21', 'test_mda_accuracy_naive_21', 'test_mda_accuracy_avg_21', 'test_mda_accuracy_sma_56_21',
# #                    'test_mda_accuracy_sma_28_21', 'test_mda_accuracy_sma_21_21', 'test_mda_accuracy_sma_14_21',
# #        'test_mda_accuracy_sma_7_21', 'test_mda_accuracy_lag_7_21', 'test_mda_accuracy_lag_14_21', 'test_mda_accuracy_lag_21_21',
# #                          'test_mda_accuracy_lag_30_21', 'test_mda_accuracy_lag_90_21', 'test_mda_accuracy_lag_365_21']].max(axis=1)
# # res['max_mda_value_28'] = res[['test_mda_accuracy_rf_28', 'test_mda_accuracy_naive_28', 'test_mda_accuracy_avg_28', 'test_mda_accuracy_sma_56_28',
# #                    'test_mda_accuracy_sma_28_28', 'test_mda_accuracy_sma_21_28', 'test_mda_accuracy_sma_14_28',
# #        'test_mda_accuracy_sma_7_28', 'test_mda_accuracy_lag_7_28', 'test_mda_accuracy_lag_14_28', 'test_mda_accuracy_lag_21_28',
# #                          'test_mda_accuracy_lag_30_28', 'test_mda_accuracy_lag_90_28', 'test_mda_accuracy_lag_365_28']].max(axis=1)
# # res['max_mda_value_35'] = res[['test_mda_accuracy_rf_35', 'test_mda_accuracy_naive_35', 'test_mda_accuracy_avg_35', 'test_mda_accuracy_sma_56_35',
# #                    'test_mda_accuracy_sma_28_35', 'test_mda_accuracy_sma_21_35', 'test_mda_accuracy_sma_14_35',
# #        'test_mda_accuracy_sma_7_35', 'test_mda_accuracy_lag_7_35', 'test_mda_accuracy_lag_14_35', 'test_mda_accuracy_lag_21_35',
# #                          'test_mda_accuracy_lag_30_35', 'test_mda_accuracy_lag_90_35', 'test_mda_accuracy_lag_365_35']].max(axis=1)

# # res['max_mda_value'] = res[['max_mda_value_7'#, 'max_mda_value_14', 'max_mda_value_21', 'max_mda_value_28', 'max_mda_value_35'
# #                            ]].max(axis=1)
# res.sort_values(by='max_mda_value_7', ascending=False, inplace=True)
# res

In [129]:
# df.to_pickle('predictions_28_12.pkl')

In [130]:
# res.to_csv('top_80_04_01_df14.csv')

In [131]:
wtd_acc_test_80 = ((res['max_value_21'] * res['test_weight_sum_21']).sum())/(res['test_weight_sum_21'].sum())
print('Weighted Accuracy on test data for top 80% contributing lanes', wtd_acc_test_80)

Weighted Accuracy on test data for top 80% contributing lanes 78.2521039961483


In [52]:
# wtd_acc_test_80 = ((res['max_mda_value_14'] * res['test_weight_sum_14']).sum())/(res['test_weight_sum_14'].sum())
# print('Weighted Mean Directional Accuracy on test data for top 80% contributing lanes', wtd_acc_test_80)

In [53]:
# wtd_acc_test_80 = ((res['max_mda_value_21'] * res['test_weight_sum_21']).sum())/(res['test_weight_sum_21'].sum())
# print('Weighted Mean Directional Accuracy on test data for top 80% contributing lanes', wtd_acc_test_80)

In [54]:
# wtd_acc_test_80 = ((res['max_mda_value_28'] * res['test_weight_sum_28']).sum())/(res['test_weight_sum_28'].sum())
# print('Weighted Mean Directional Accuracy on test data for top 80% contributing lanes', wtd_acc_test_80)

In [55]:
# wtd_acc_test_80 = ((res['max_mda_value_35'] * res['test_weight_sum_7']).sum())/(res['test_weight_sum_7'].sum())
# print('Weighted Mean Directional Accuracy on test data for top 80% contributing lanes', wtd_acc_test_80)